In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
# import matplotlib.pyplot as plt
import datetime

In [90]:
f = np.load("mfcc_fixed.npz")
X, Y = f['X'], f['Y']
X_train =  X[0:900]
Y_train =  Y[0:900]        # fold 10


x_test, y_test = X[900:1000], Y[900:1000]
x_train, y_train = X_train[0:800], Y_train[0:800]
x_val, y_val= X_train[800:900], Y_train[800:900]

In [99]:
batch_size = 100

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)


x = x_train[0:batch_size]
x_rank = tf.rank(x).numpy()
x_norm_resize_shape = [batch_size] + list(tf.ones(tf.rank(x), dtype=tf.int32).numpy())[1:]

In [100]:
def resblock(inputs, num_feature_maps):
    # BLOCK 1

    conv_x = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=3, padding='same')(inputs)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=3, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)


    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=num_feature_maps, kernel_size=1, padding='same')(inputs)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    outputs = keras.layers.add([shortcut_y, conv_y])
    outputs = keras.layers.Activation('relu')(outputs)
    return outputs

In [101]:
def build_model(input_shape, batch_size):
    inputs = keras.layers.Input(shape=input_shape, batch_size=32)

    layer1 = resblock(inputs, 64)
    layer1 = resblock(layer1, 64)

    layer2 = resblock(layer1, 128)
    layer2 = resblock(layer2, 128)

    layer3 = resblock(layer1, 256)
    layer3 = resblock(layer2, 256)

    layer3 = resblock(layer1, 512)
    layer3 = resblock(layer2, 512)

    pool = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(layer3)

    output = keras.layers.Dense(5, activation='softmax')(pool)



    return keras.models.Model(inputs=inputs, outputs=output)

In [102]:
model = build_model(X[0].shape, 32)

In [103]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(32, 54, 30)]               0         []                            
                                                                                                  
 conv1d_320 (Conv1D)         (32, 54, 64)                 5824      ['input_11[0][0]']            
                                                                                                  
 batch_normalization_320 (B  (32, 54, 64)                 256       ['conv1d_320[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_240 (Activation  (32, 54, 64)                 0         ['batch_normalization_3

In [104]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
lds = lambda x, y: tf.math.reduce_sum(keras.losses.kl_divergence(x, y))
acc_metric = keras.metrics.SparseCategoricalAccuracy()
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, weight_decay=0.0001)
rLRoP_callback = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                    factor=0.1,
                                                    patience=10,
                                                    verbose=0,
                                                    mode="auto",
                                                    min_delta=0.0001,
                                                    cooldown=0,
                                                    min_lr=0.0)
es_callback = keras.callbacks.EarlyStopping(
                                            monitor="val_loss",
                                            min_delta=0,
                                            patience=100,
                                            verbose=1,
                                            mode="auto",
                                            baseline=None,
                                            restore_best_weights=True,
                                            start_from_epoch=0)
                                                    
model.compile(optimizer=optimizer, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [105]:
model.fit(train_dataset, validation_data=val_dataset, batch_size=batch_size, epochs=2000,
          callbacks=[es_callback, rLRoP_callback])

Epoch 1/2000
8/8 [==============================] - 7s 78ms/step - loss: 1.7306 - accuracy: 0.3925 - val_loss: 1.7236 - val_accuracy: 0.2800 - lr: 0.1000
Epoch 2/2000
8/8 [==============================] - 0s 19ms/step - loss: 0.7126 - accuracy: 0.7462 - val_loss: 5.1480 - val_accuracy: 0.2000 - lr: 0.1000
Epoch 3/2000
8/8 [==============================] - 0s 19ms/step - loss: 0.3154 - accuracy: 0.8925 - val_loss: 6.6804 - val_accuracy: 0.4300 - lr: 0.1000
Epoch 4/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.1642 - accuracy: 0.9413 - val_loss: 11.9481 - val_accuracy: 0.3800 - lr: 0.1000
Epoch 5/2000
8/8 [==============================] - 0s 19ms/step - loss: 0.1112 - accuracy: 0.9638 - val_loss: 11.3483 - val_accuracy: 0.3700 - lr: 0.1000
Epoch 6/2000
8/8 [==============================] - 0s 19ms/step - loss: 0.0786 - accuracy: 0.9775 - val_loss: 7.7935 - val_accuracy: 0.3800 - lr: 0.1000
Epoch 7/2000
8/8 [==============================] - 0s 18ms/step - loss: 0

In [118]:
acc_metric.reset_states()
acc_metric.update_state(y_test, model(x_test))
acc = acc_metric.result().numpy()
loss = loss_fn(y_test, model(x_test)).numpy()
print(acc, loss)

1.0 0.010352697
